In [1]:
print(9)


9


In [11]:
%pwd

'C:\\Users\\kentj\\HealthMate\\backend'

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
#Extract data from the PDF
def load_pdf_file(pdf):
    loader = DirectoryLoader(pdf,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents


In [14]:
extracted_data = load_pdf_file(pdf="pdf/")


In [7]:
#extracted_data

In [15]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [16]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [10]:
#text_chunks

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

In [20]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [21]:
embeddings = download_hugging_face_embeddings()

In [22]:
%pwd

'C:\\Users\\kentj\\HealthMate\\backend'

In [24]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [27]:
#query_result

In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
import os
PINECONE_API_KEY= os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY= os.environ.get("OPENAI_API_KEY")


In [34]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec


pc= Pinecone(api_key=PINECONE_API_KEY)

index_name="healthmate"


pc.create_index(
    name=index_name,
    dimension=384,  #  must match the embedding size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
print(f"Created index '{index_name}' with dimension 384")


Created index 'healthmate' with dimension 384


In [47]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [36]:
# 3. Embed each chunk and upsert the embeddings to my Pinecone index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


C:\Users\kentj\HealthMate\backend\env\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [37]:
#Load existing index
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert them embeddings into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings, 
)


In [38]:
docsearch

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [41]:
retrieved_docs = retriever.invoke("What is Acne?")

C:\Users\kentj\HealthMate\backend\env\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [42]:
retrieved_docs

[Document(id='f60b0cce-eff0-4a61-b708-d4e3d0401cc4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'pdf\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='dcd7f9aa-c025-4b43-b09a-d6c4aa78e64c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'pdf\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 

In [48]:
from langchain_openai import OpenAI
llm= OpenAI(temperature=0.4, max_tokens=500)

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assiatant for question-answering tasks."
    "Use the following pieces of retrived context to answer "
    "the question.if you don't know the answer, say that  you "
    "don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)




In [50]:
question_answer_chain= create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain (retriever,question_answer_chain)

In [51]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])


C:\Users\kentj\HealthMate\backend\env\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)




Acne is a common skin disease that causes pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It is the most common skin disease in the United States, affecting approximately 17 million people.


In [55]:
response = rag_chain.invoke({"input": "What is a phone?"})
print(response["answer"])


C:\Users\kentj\HealthMate\backend\env\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)




I don't know.


In [56]:
response = rag_chain.invoke({"input": "What is the treatment for acne ?"})
print(response["answer"])


C:\Users\kentj\HealthMate\backend\env\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)




The treatment for acne depends on the severity, with mild cases being treated with topical drugs like tretinoin, benzoyl peroxide, adapalene, or salicylic acid. Inflammation can be addressed with topical antibiotics. Improvement is usually seen in 2-4 weeks. For more severe cases, oral medication like isotretinoin may be prescribed, with improvement taking 2 or more months. Acne can reappear when treatment stops, but may improve over time. Prevention methods include gentle washing, avoiding abrasive cleansers, and using noncomedogenic makeup and moisturizers.
